In [85]:
import csv
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as bs
from time import sleep
from bs4 import SoupStrainer
import requests
import lxml
import random
from datetime import datetime
from itertools import cycle
import re

In [86]:
def get_ips():
    ip_list = []
    with open('myproxylist.txt', 'r') as file:
        for line in file:
            proxy = line.strip()
            ip = proxy.split(':')[0]
            ip_list.append(ip)
    return ip_list

In [117]:
def append_dict_as_row(file_name, dict_of_elem):
    # Specify field names
    field_names = ['Url', 'Headline', 'ProxyIP', 'Publication Date', 'Content', 'Image URL']
    
    # Open the CSV file in append mode and create a CSV writer object
    with open(file_name, 'a', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.DictWriter(csvfile, fieldnames=field_names)
        
        # Check if the file is empty (no existing header)
        file_empty = csvfile.tell() == 0
        
        # If the file is empty, write the header row
        if file_empty:
            csvwriter.writeheader()
        
        # Write the values of the dictionary as a row in the CSV file
        csvwriter.writerow(dict_of_elem)

In [131]:
count = 1
ip_list = get_ips()

# use a proxy pool and cycle  
proxy_pool = cycle(ip_list)

global proxy_pool

proxy = next(proxy_pool)

def fetch_news_90Min(link):

    global count
    
    global proxy
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"}
    response = requests.get(link, headers=headers, proxies={"http": proxy})
    strained = SoupStrainer('div', class_='horizontalCardsGrid_bjqrxn')
    
    soup = bs(response.content, 'lxml', parse_only=strained, from_encoding='utf-8')
    
    for links in soup.find_all('article', class_='style_teed7n-o_O-wrapper_1wgo221'):
        a_tag = links.find('a')['href']
        
        image = links.find('figure', class_='base_1kci93k')
        image = str(image)
        
        pattern = r'srcset="([^"]*)"'
        
        matches = re.search(pattern, image)

        if matches:
            srcset_attribute = matches.group(1)
            image = srcset_attribute.split()
            image = image[4].split(' ')[0] if len(image) > 3 else None
        else:
            image = 'Not found'
        
        para = links.find('p', class_='tagStyle_z4kqwb-o_O-style_e4x16a-o_O-style_1q4n9dr').text
        
        date = links.find('h5', class_='tagStyle_bplnm6-o_O-style_1ju2kci').find('time').text
        
        headline = links.find('h3', class_='tagStyle_1igopqi-o_O-style_e4x16a-o_O-text_t8zpz8').text
        
        print(f'Article Number: {count}, Headline: {headline} Image URL: {image}, ProxyIP: {proxy} , Publication Date: {date}, Link: {a_tag}, content: {para}')
        append_dict_as_row('dataset_90Min.csv', {'Url': a_tag, 'Headline': headline, 'ProxyIP': proxy, 'Publication Date': date, 'Content': para, 'Image URL': image})
        count += 1

In [132]:
base_url = 'https://www.90min.com/players/lionel-messi?page='

no_of_pages = 27

for pages in range(1, no_of_pages+1):
    
    link = base_url + str(pages)
    
    fetch_news_90Min(link)
    
    proxy = next(proxy_pool)
    
    sleep(3)
    
    print(f'Page: {pages} done')

Article Number: 1, Headline: Lionel Messi vs Cristiano Ronaldo: Complete head-to-head record Image URL: https://images2.minutemediacdn.com/image/upload/c_crop,w_2310,h_1299,x_0,y_0/c_fill,w_540,ar_16:9,f_auto,q_auto,g_auto/images%2FGettyImages%2Fmmsport%2F90min_en_international_web%2F01hmxxxxz1hzvgja570x.jpg, ProxyIP: 38.154.227.167 , Publication Date: Jan 26, 2024, Link: https://www.90min.com/posts/lionel-messi-vs-cristiano-ronaldo-complete-head-to-head-record, content: The complete head-to-head record between Lionel Messi and Cristiano Ronaldo in what is the greatest football rivalry of all time. 
Article Number: 2, Headline: Cristiano Ronaldo insists Ballon d'Or and The Best awards are 'losing credibility' after Lionel Messi wins Image URL: https://images2.minutemediacdn.com/image/upload/c_crop,w_4898,h_2755,x_0,y_123/c_fill,w_540,ar_16:9,f_auto,q_auto,g_auto/images%2FGettyImages%2Fmmsport%2F90min_en_international_web%2F01hmpb4vcv2zkcp8pmav.jpg, ProxyIP: 38.154.227.167 , Publication